# extract feats

In [2]:
import os
import sys
sys.path.append('/workspace/GREAT_ASV_system/train/')

import numpy as np
import torch


import training_utils
from torch.utils.data import *
from my_dataloader import *
from read_data import *

from model_bank import *
from models import *


In [3]:
component_dir = '/workspace/LOGS_OUTPUT/std_server5/backend/vox&sdsv&libri_xvector_1L_3DO_FALDA'

SCORING_COSINE_NAME = 'score_cosine'
ENR_DATA_NAME = 'enr_data'
EVL_DATA_NAME = 'evl_data'


In [4]:
if not os.path.isdir(component_dir):
    os.makedirs(component_dir)

# GPU multiprocess T1

In [5]:
model_path = '/Lun2/rzz/kaldi-master/egs/zhiyong/sdsvc_exp/train_exp/full mix 1L FOCAL_ALDA_OPT_FAST_3DO1/ckpt/sdsvc_metric_saver_MINC.model'
model_id = ''
model_metric = ''

In [6]:
def extract_feature_m(i, train_data, class_list_new_m):
#     model_settings = {'in_feat': 30, 'emb_size': 512, 'class_num': 7811, 'source_class_num': 7323, \
#     's': 50, 'm': 0.2, 'anneal_steps': 10000, 'weight': 12, 'focal_d_gamma': 0, 'focal_al_gamma': 0}
#     model_settings = {'in_feat': 30, 'emb_size': 512, 'class_num': 8983, 'class_split': [0, 7323, 7811, 8983], \
#     's': 50, 'm': 0.2, 'anneal_steps': 10000, 'weight_split': [15, 1, 2.4], 'focal_d_gamma': 0, 'focal_al_gamma': 0}
    
    model_settings = {'in_feat': 30, 'emb_size': 512, 'class_num': 8983, 'class_split': [0, 8495, 8983], \
    's': 50, 'm': 0.2, 'anneal_steps': 10000, 'weight_split': [17.4, 1], 'focal_d_gamma': 0, 'focal_al_gamma': 0}    
    torch.backends.cudnn.benchmark = False

    os.environ['CUDA_VISIBLE_DEVICES'] = str(i%2)

    device = torch.device("cuda:0")

    train_dataloader = My_DataLoader(train_data, batch_size=None, shuffle=False, sampler=None,\
    batch_sampler=None, num_workers=8, collate_fn=None,\
    pin_memory=False, drop_last=False, timeout=0,\
    worker_init_fn=None, multiprocessing_context=None)

#     model = FOCAL_ALDA.FOCAL_ALDA_2DO_OPT_FAST(model_settings)
    model = FOCAL_ALDA.FOCAL_ALDA_MULDO_OPT_FAST(model_settings)
    checkpoint = torch.load(model_path, map_location='cpu')
    model.load_state_dict(checkpoint['model'], strict=True)
    model = model.to(device)
    model.eval()

    class_list_new = {}

    for count, (batch_x, batch_y) in enumerate(train_dataloader):
        torch.cuda.empty_cache()
        batch_x = batch_x.to(device)
        label = batch_y[0].split('-')[0]
        batch_y = torch.tensor([0]).to(device)
        try:
            with torch.no_grad():
                _, _, emb, _, _ = model(batch_x, batch_y, mod='eval')
        except:
            print('Proc', str(i), 'EER:', label)
            continue
    #     _, _, emb, _, _ = model(batch_x, batch_y, mod='eval')

        emb = emb.squeeze().data.cpu().numpy()

        if label not in class_list_new.keys():
            class_list_new[label] = emb[None, :]
        else:
            class_list_new[label] = np.append(class_list_new[label], emb[None, :], axis=0)

        if (count+1) % 10000 == 0:
            print('Proc '+ str(i) + ':' + str((count+1) // 10000))
    
    class_list_new_m[i] = class_list_new
    del model, batch_x, batch_y
    torch.cuda.empty_cache()

# GPU multiprocess T2

In [6]:
model_path = '/Lun2/rzz/kaldi-master/egs/zhiyong/sdsvc_exp/train_exp/new_pipe_test_fullmix 1L 3DO/ckpt/vox1test_metric_saver_MINC.model'
model_id = 'Xvector_SAP_1L'
model_metric = 'AM_normfree_softmax_anneal_ce_head'

In [7]:
def extract_feature_m(i, train_data, class_list_new_m):
    model_settings = {'in_feat': 30, 'emb_size': 512, 'class_num': 8983, \
    's': 50, 'm': 0.2, 'anneal_steps': 10000}
    torch.backends.cudnn.benchmark = False

    os.environ['CUDA_VISIBLE_DEVICES'] = str(i%2)

    device = torch.device("cuda:0")

    train_dataloader = My_DataLoader(train_data, batch_size=None, shuffle=False, sampler=None,\
    batch_sampler=None, num_workers=8, collate_fn=None,\
    pin_memory=False, drop_last=False, timeout=0,\
    worker_init_fn=None, multiprocessing_context=None)

    model = get_model(model_id, model_metric, model_settings, None)
    checkpoint = torch.load(model_path, map_location='cpu')
    model.load_state_dict(checkpoint['model'], strict=True)
    model = model.to(device)
    model.eval()

    class_list_new = {}

    for count, (batch_x, batch_y) in enumerate(train_dataloader):
        torch.cuda.empty_cache()
        batch_x = batch_x.to(device)
        label = batch_y[0].split('-')[0]
        batch_y = torch.tensor([0]).to(device)
        try:
            with torch.no_grad():
                _, _, emb, _, _ = model(batch_x, batch_y, mod='eval')
        except:
            print('Proc', str(i), 'EER:', label)
            continue
    #     _, _, emb, _, _ = model(batch_x, batch_y, mod='eval')

        emb = emb.squeeze().data.cpu().numpy()

        if label not in class_list_new.keys():
            class_list_new[label] = emb[None, :]
        else:
            class_list_new[label] = np.append(class_list_new[label], emb[None, :], axis=0)

        if (count+1) % 10000 == 0:
            print('Proc '+ str(i) + ':' + str((count+1) // 10000))
    
    class_list_new_m[i] = class_list_new
    del model, batch_x, batch_y
    torch.cuda.empty_cache()

# GPU multiprocess for enr

In [9]:
from multiprocessing import Process, Manager

In [10]:
num_p = 10

In [11]:
train_list = '/workspace/DATASET/std/sdsv20_lb_eval/enr_mfcc.csv'
train_data = PickleDataSet(train_list)

In [12]:
train_data_len = len(train_data)
num_per_process = (train_data_len // num_p) + 1

In [13]:
train_data_len

110673

In [14]:
manager = Manager()
class_list_new_m = manager.list()

In [15]:
data_m = []
# class_list_new_m = []
for i in range(num_p):
    data = torch.utils.data.Subset(train_data, np.arange(i*num_per_process, min((i+1)*num_per_process, train_data_len)))
    data_m.append(data)
    class_list_new_m.append({})
    print(len(data))

11068
11068
11068
11068
11068
11068
11068
11068
11068
11061


In [16]:
processes = [Process(target = extract_feature_m, args = (i, data_m[i], class_list_new_m)) for i in range(num_p)]
[p.start() for p in processes]
joined = [p.join() for p in processes]

Proc 4:1
Proc 6:1
Proc 2:1
Proc 8:1
Proc 0:1
Proc 1:1
Proc 5:1
Proc 9:1
Proc 3:1
Proc 7:1


In [17]:
count = 0
for i in class_list_new_m:
    for j in i:
        count += len(i[j])
count

110673

In [18]:
class_list_new = class_list_new_m[0]

for count, this_list in enumerate(class_list_new_m):
    if count == 0:
        continue
    for this_label in this_list:
        if this_label not in class_list_new.keys():
            class_list_new[this_label] = this_list[this_label]
        else:
            class_list_new[this_label] = np.append(class_list_new[this_label], this_list[this_label], axis=0)

In [19]:
count = 0
for i in class_list_new:
    count += len(class_list_new[i])
count

110673

In [20]:
nanlist = []
for i in class_list_new:
    if np.isnan(class_list_new[i]).any():
        print(i)
        nanlist.append(i)
for i in nanlist:
    class_list_new.pop(i)

In [21]:
count

110673

In [22]:
len(class_list_new)

110673

In [23]:
import pickle
out = component_dir+'/'+ENR_DATA_NAME
with open(out, 'wb') as handle:
    pickle.dump(class_list_new, handle)

# GPU multiprocess for evl

In [24]:
from multiprocessing import Process, Manager

In [25]:
num_p = 10

In [26]:
train_list = '/workspace/DATASET/std/sdsv20_lb_eval/evl_mfcc.csv'
train_data = PickleDataSet(train_list)

In [27]:
train_data_len = len(train_data)
num_per_process = (train_data_len // num_p) + 1

In [28]:
train_data_len

69542

In [29]:
manager = Manager()
class_list_new_m = manager.list()

In [30]:
data_m = []
# class_list_new_m = []
for i in range(num_p):
    data = torch.utils.data.Subset(train_data, np.arange(i*num_per_process, min((i+1)*num_per_process, train_data_len)))
    data_m.append(data)
    class_list_new_m.append({})
    print(len(data))

6955
6955
6955
6955
6955
6955
6955
6955
6955
6947


In [31]:
processes = [Process(target = extract_feature_m, args = (i, data_m[i], class_list_new_m)) for i in range(num_p)]
[p.start() for p in processes]
joined = [p.join() for p in processes]

In [32]:
count = 0
for i in class_list_new_m:
    for j in i:
        count += len(i[j])
count

69542

In [33]:
class_list_new = class_list_new_m[0]

for count, this_list in enumerate(class_list_new_m):
    if count == 0:
        continue
    for this_label in this_list:
        if this_label not in class_list_new.keys():
            class_list_new[this_label] = this_list[this_label]
        else:
            class_list_new[this_label] = np.append(class_list_new[this_label], this_list[this_label], axis=0)

In [34]:
count = 0
for i in class_list_new:
    count += len(class_list_new[i])
count

69542

In [35]:
nanlist = []
for i in class_list_new:
    if np.isnan(class_list_new[i]).any():
        print(i)
        nanlist.append(i)
for i in nanlist:
    class_list_new.pop(i)

In [36]:
count

69542

In [37]:
len(class_list_new)

69542

In [38]:
import pickle
out = component_dir+'/'+EVL_DATA_NAME
with open(out, 'wb') as handle:
    pickle.dump(class_list_new, handle)

# Enroll models

In [39]:
import pickle
with open(component_dir+'/'+ENR_DATA_NAME, 'rb') as handle:
    enr_data = pickle.load(handle)

In [40]:
enr_list = {}
num_utt = {}
with open('/workspace/DATASET/std/sdsv20_lb_eval/model_enrollment.txt', 'r') as f:
    for count, line in enumerate(f):
        if count == 0:
            continue
        info = line[:-1].split(' ')
        model_label = info[0]
        num_utt[model_label] = len(info)-1
        for i in range(1, len(info)):
            if model_label not in enr_list.keys():
                enr_list[model_label] = enr_data[info[i]]
            else:
                enr_list[model_label] = np.append(enr_list[model_label], enr_data[info[i]], axis=0)

# Test data

In [41]:
import pickle
with open(component_dir+'/'+EVL_DATA_NAME, 'rb') as handle:
    evl_data = pickle.load(handle)

# Cosine Scoring

In [42]:
trail_path = '/workspace/DATASET/std/sdsv20_lb_eval/trials.txt'
score_out_path = component_dir+'/'+SCORING_COSINE_NAME

In [43]:
for i in enr_list:
    enr_list[i] = np.mean(enr_list[i], axis=0).squeeze()
    enr_list[i] = (1.0 / np.linalg.norm(enr_list[i])) * enr_list[i]

In [44]:
for i in evl_data:
    evl_data[i] = evl_data[i].squeeze()
    evl_data[i] = (1.0 / np.linalg.norm(evl_data[i])) * evl_data[i]

In [45]:
with open(score_out_path, 'w') as of:
    with open(trail_path, 'r') as f:
        for count, line in enumerate(f):
            line = line[:-1]
            if count == 0:
                print(line)
                continue
            enroll_emb = enr_list[line.split(' ')[0]].squeeze()
            test_emb = evl_data[line.split(' ')[1]].squeeze()

            cosine = np.dot(enroll_emb, test_emb)
            
            of.write('{:.3f}'.format(cosine)+'\n')
            
            if (count+1) % 100000 == 0:
                print((count+1) // 100000)

model-id evaluation-file-id
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
